In [9]:
#!pip install neo4j
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import graph_model as gm
np.set_printoptions(suppress=True)

In [10]:
uri = "bolt://3.220.233.169:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "i-0e23d19f0d8795714"))

In [ ]:
gm.graph_model_predict(driver, 'YQwAvXUkeUpBat32FU_4eQ', '_CEo5qKtrYIOH6BTxnQWHQ','PA')

In [3]:
gm.user_preference(driver, 'YQwAvXUkeUpBat32FU_4eQ', '_CEo5qKtrYIOH6BTxnQWHQ')

array([0.00014861, 0.01494001, 0.80208579, 0.16788557, 0.01494001])

In [ ]:
gm.biz_preference(driver, 'YQwAvXUkeUpBat32FU_4eQ', '_CEo5qKtrYIOH6BTxnQWHQ')

In [3]:
pa_reviews=pd.read_pickle('PA_reviews')

In [5]:
test_sample=pa_reviews.sample(100)

In [6]:
test_sample.head()

,userID,rating,itemID
259986,jJV8iMLq-hg0-Cdxgfa0WA,5.0,hsRjct8UiUHU54B983GmcA
195918,nRXOxMrr19AP8P_fYA9RTw,2.0,CwcDRtNgjgM4pLS4o63g6A
139838,F58wGIoXK-FPLb9YrSA9SQ,1.0,9IhBQ7mTYajCQtIrEjv5KQ
248958,6Ki3bAL0wx9ymbdJqbSWMA,3.0,MyITm7hhBUqRmz2PqlfhOw
277677,cA8_7apjrrZouRuVwNZgFQ,5.0,hzumXofKoqICLsHETI1LAA


In [ ]:
test_sample['prediction']=[gm.graph_model_predict(driver, x , y ,'PA') for x,y in zip(test_sample['userID'], test_sample['itemID'])]

In [15]:
olduser_id=195103
oldbiz_id=17

user_id='JSrP-dUmLlwZiI7Dp3PQ2A'
biz_id='PZ-LZzSlhSe9utkQYU8pFg'

In [5]:
def cypher(driver, query, results_columns):
    """input is a neo4j connection driver, a string representing a cypher query
    and a list of string for data frame column names. returns the dataframe of the results."""
    
    with driver.session() as session:
        result = session.run(query)
        
    result_df = pd.DataFrame(result.values() , columns=results_columns)
    
    return result_df

In [10]:
#testing cypher function
review_dist=cypher(driver, f"\
        MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business)\
        WHERE u.id='{user_id}' RETURN r.id, r.stars, b.id LIMIT 10" , ['r.id','r.stars','b.id'])
review_dist.head(14)

,r.id,r.stars,b.id
0,6BnQwlxRn7ZuWdzninM9sQ,3.0,cHdJXLlKNWixBXpDwEGb_A
1,u95ZUYzRcW80xiv9MzUqZA,5.0,Xg5qEQiB-7L6kGJ5F4K3bQ
2,s60cMfmdj2pWXQxUhYG6ZA,4.0,65GMvjjrsvyR9pxxAys7qg
3,OIejtNr6eQQBANKa-YwaZw,5.0,EneqsOFGCK4KXofNHsc5aQ
4,GW8BAKQPH9eoONr2MK-T1w,4.0,faPVqws-x-5k2CQKDNtHxw
5,smWh1nC1R66Tr-L5hNZg4w,5.0,qTmuW0C9XXuxWwumvn89sw
6,fcmj1_VQ8_epSxH-oHTqMw,4.0,yNPh5SO-7wr8HPpVCDPbXQ
7,RLDjVoKkWA1qRTZxbfoozA,2.0,HEmSK3apoP4xwVs0YkpNLQ
8,4mcL8k2bh3Wy6-gXv4Poag,3.0,AD5r187-LD836JW6ERa9Cg
9,9VqZYuARpPIVCrWmB8Xltw,5.0,zXvtouoaOsL0Bp0udugAjQ


49

In [5]:
def user_preference_dist(user_id ,biz_id):

    review_dist=cypher(driver, f"\
    MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business)\
    WHERE u.id='{user_id}' RETURN r.id, r.stars, b.id", ['r.id','r.stars','b.id'])

    
    biz_categories=cypher(driver, f"\
    MATCH (b:Business)-[:IN_CATEGORY]->(c:Category)\
    WHERE b.id='{biz_id}' RETURN c.id", ['c.id'])
    
    review_stars=review_dist['r.stars'].value_counts()
    mean_review=review_dist['r.stars'].mean()
    num_reviews=review_dist['r.stars'].shape[0]
    cat_ids=list(biz_categories['c.id'])
    
    

    biz_in_cat=[]
    for cat in cat_ids:
        temp=cypher(driver, f"MATCH (b:Business)-[:IN_CATEGORY]->(c:Category)\
        WHERE c.id='{cat}' AND NOT b.id='{biz_id}' RETURN b.id", ['b.id'])
        biz_in_cat.append(temp)
        
    
   
    reviews_in_cat=[]
    for i in range(len(biz_in_cat)):
        
        sim_biz = cypher(driver, f"MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business) WHERE u.id='{user_id}' and b.id IN {list(biz_in_cat[i]['b.id'])}\
        RETURN r.stars, b.id", ['r.stars','b.id'])
        #print(list(biz_in_cat[i]['b.id']))
        reviews_in_cat.append(sim_biz)

    numerator=np.empty(5)
    for i in (1,2,3,4,5):
        try:
            numerator[i-1]=review_stars[i]
        except:
            numerator[i-1]=0

    PRu=(numerator+1)/(num_reviews+5)

    num_cat=len(biz_in_cat)
    cats_by_stars=np.empty((num_cat,5))

    for i in range(num_cat):
        cat_stars=reviews_in_cat[i]['r.stars'].value_counts()
        for j in (1,2,3,4,5):
            try:
                cats_by_stars[i][j-1]=cat_stars[j]
            except:
                cats_by_stars[i][j-1]=0

    PRaj=( (cats_by_stars+1) / (numerator+num_cat) ).prod(axis=0)

    user_prefs_un_normalized=PRu*PRaj

    user_prefs=user_prefs_un_normalized/sum(user_prefs_un_normalized)
    
    return user_prefs

In [104]:
user_preference_dist(user_id ,biz_id)

array([0.07703953, 0.14791589, 0.15339426, 0.16101261, 0.46063772])

In [6]:
def biz_acceptance(user_id ,biz_id):
    
    review_dist=cypher(driver, f"\
    MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business)\
    WHERE b.id='{biz_id}' RETURN r.id, r.stars, u.id", ['r.id','r.stars','u.id'])
    
    user_categories=cypher(driver, f"\
    MATCH (u:User)-[]->(r:Reputation)\
    WHERE u.id='{user_id}' RETURN r.id", ['r.id'])
    
    
    review_stars=review_dist['r.stars'].value_counts()
    mean_review=review_dist['r.stars'].mean()
    num_reviews=review_dist['r.stars'].shape[0]
    cat_ids=list(user_categories['r.id'])
    
    
    
    user_in_cat=[]
    
    for cat in cat_ids:
        temp=cypher(driver, f"MATCH (u:User)-[]->(r:Reputation)\
        WHERE r.id ='{cat}' AND NOT u.id='{user_id}'  RETURN u.id",['u.id'])
        user_in_cat.append(temp)
        
        
    reviews_in_cat=[]
    for i in range(len(user_in_cat)):
        #replace this with some sort of join, merge, intersection whatever with reivew dist and biz in cat
        sim_user = cypher(driver, f"MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business) WHERE b.id='{biz_id}' and u.id IN {list(user_in_cat[i]['u.id'])}\
        RETURN r.stars, u.id", ['r.stars','u.id'])
        reviews_in_cat.append(sim_user)

    numerator=np.empty(5)
    for i in (1,2,3,4,5):
        try:
            numerator[i-1]=review_stars[i]
        except:
            numerator[i-1]=0

    PRu=(numerator+1)/(num_reviews+5)

    num_cat=len(user_in_cat)
    cats_by_stars=np.empty((num_cat,5))

    for i in range(num_cat):
        cat_stars=reviews_in_cat[i]['r.stars'].value_counts()
        for j in (1,2,3,4,5):
            try:
                cats_by_stars[i][j-1]=cat_stars[j]
            except:
                cats_by_stars[i][j-1]=0

    PRaj=( (cats_by_stars+1) / (numerator+num_cat) ).prod(axis=0)

    biz_prefs_un_normalized=PRu*PRaj

    biz_prefs=biz_prefs_un_normalized/sum(biz_prefs_un_normalized)
    
    return biz_prefs

In [97]:
biz_acceptance(user_id ,biz_id)

array([0.02246682, 0.0780098 , 0.10234522, 0.3173092 , 0.47986895])

In [6]:
def expected_rating(rating_dist):
    runsum=0
    for i in [1,2,3,4,5]:
        runsum+=rating_dist[i-1]*i
    return runsum

In [81]:
user = user_preference_state('YQwAvXUkeUpBat32FU_4eQ', '_CEo5qKtrYIOH6BTxnQWHQ','PA')

In [82]:
biz = biz_preference_state('YQwAvXUkeUpBat32FU_4eQ', '_CEo5qKtrYIOH6BTxnQWHQ','PA')

In [83]:
user,biz

(array([0.00014861, 0.01494001, 0.80208579, 0.16788557, 0.01494001]),
 array([0.13100221, 0.04314627, 0.05265675, 0.13364174, 0.63955303]))

In [84]:
expected_rating(user)

3.1825283562770643

In [85]:
expected_rating(biz)

4.107597113477149

In [86]:
expected_rating((biz*user)/sum((biz*user)))

3.5456325999488554

In [80]:
def biz_preference_state(user_id ,biz_id,state):
    
    review_dist=cypher(driver, f"\
    MATCH (s:State)<-[:REVIEWS_IN]-(u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business)\
    WHERE b.id='{biz_id}' AND s.name='{state}' RETURN r.id, r.stars, u.id", ['r.id','r.stars','u.id'])
    
    user_categories=cypher(driver, f"\
    MATCH (u:User)-[]->(r:Reputation)\
    WHERE u.id='{user_id}' RETURN r.id", ['r.id'])
    
    
    review_stars=review_dist['r.stars'].value_counts()
    mean_review=review_dist['r.stars'].mean()
    num_reviews=review_dist['r.stars'].shape[0]
    cat_ids=list(user_categories['r.id'])
    
    
    
    user_in_cat=[]
    
    for cat in cat_ids:
        temp=cypher(driver, f"MATCH (s:State)<-[:REVIEWS_IN]-(u:User)-[]->(r:Reputation)\
        WHERE r.id ='{cat}' AND NOT u.id='{user_id}' AND s.name='{state}' RETURN u.id",['u.id'])
        user_in_cat.append(temp)
        
        
    reviews_in_cat=[]
    for i in range(len(user_in_cat)):
        #replace this with some sort of join, merge, intersection whatever with reivew dist and biz in cat
        sim_user = cypher(driver, f"MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business) WHERE b.id='{biz_id}' and u.id IN {list(user_in_cat[i]['u.id'])}\
        RETURN r.stars, u.id", ['r.stars','u.id'])
        reviews_in_cat.append(sim_user)
    
    numerator=np.empty(5)
    for i in (1,2,3,4,5):
        try:
            numerator[i-1]=review_stars[i]
        except:
            numerator[i-1]=0

    PRu=(numerator+1)/(num_reviews+5)

    num_cat=len(user_in_cat)
    cats_by_stars=np.empty((num_cat,5))

    for i in range(num_cat):
        if len(reviews_in_cat[i])!=0:
            cat_stars=reviews_in_cat[i]['r.stars'].value_counts()
            for j in (1,2,3,4,5):
                try:
                    cats_by_stars[i][j-1]=cat_stars[j]
                except:
                    cats_by_stars[i][j-1]=0
                    
        else:
            
            for j in (1,2,3,4,5):
                try:
                    cats_by_stars[i][j-1]=review_stars[j]
                except:
                    cats_by_stars[i][j-1]=0

    PRaj=( (cats_by_stars+1) / (numerator+num_cat) ).prod(axis=0)

    biz_prefs_un_normalized=PRu*PRaj

    biz_prefs=biz_prefs_un_normalized/sum(biz_prefs_un_normalized)
    
    return biz_prefs

In [75]:
def user_preference_state(user_id ,biz_id,state):

    review_dist=cypher(driver, f"\
    MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business)-[:IN_CITY]->(:City)-[:IN_STATE]->(s:State)\
    WHERE u.id='{user_id}'  AND s.name='{state}' RETURN r.id, r.stars, b.id", ['r.id','r.stars','b.id'])

    
    biz_categories=cypher(driver, f"\
    MATCH (b:Business)-[:IN_CATEGORY]->(c:Category) \
    WHERE b.id='{biz_id}' RETURN c.id", ['c.id'])
    
    review_stars=review_dist['r.stars'].value_counts()
    
    mean_review=review_dist['r.stars'].mean()
    num_reviews=review_dist['r.stars'].shape[0]
    cat_ids=list(biz_categories['c.id'])
    
    

    biz_in_cat=[]
    for cat in cat_ids:
        
        temp=cypher(driver, f"\
        MATCH (s:State)<-[:IN_STATE]-(:City)<-[:IN_CITY]-(b:Business)-[:IN_CATEGORY]->(c:Category) \
        WHERE c.id='{cat}' AND NOT b.id='{biz_id}' AND s.name='{state}' RETURN b.id", ['b.id'])
        biz_in_cat.append(temp)
        
    reviews_in_cat=[]
    for i in range(len(biz_in_cat)):
        
        sim_biz = cypher(driver, f"MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business) WHERE u.id='{user_id}' and b.id IN {list(biz_in_cat[i]['b.id'])}\
        RETURN r.stars, b.id", ['r.stars','b.id'])
        
        reviews_in_cat.append(sim_biz)
        
    
    numerator=np.empty(5)
    for i in (1,2,3,4,5):
        try:
            numerator[i-1]=review_stars[i]
        except:
            numerator[i-1]=0

    PRu=(numerator+1)/(num_reviews+5)

    num_cat=len(biz_in_cat)
    cats_by_stars=np.empty((num_cat,5))

    for i in range(num_cat):
        if len(reviews_in_cat[i])!=0:
            cat_stars=reviews_in_cat[i]['r.stars'].value_counts()
            for j in (1,2,3,4,5):
                try:
                    cats_by_stars[i][j-1]=cat_stars[j]
                except:
                    cats_by_stars[i][j-1]=0
        else:
            for j in (1,2,3,4,5):
                try:
                    cats_by_stars[i][j-1]=review_stars[j]
                except:
                    cats_by_stars[i][j-1]=0
            
    PRaj=( (cats_by_stars+1) / (numerator+num_cat) ).prod(axis=0)

    user_prefs_un_normalized=PRu*PRaj

    user_prefs=user_prefs_un_normalized/sum(user_prefs_un_normalized)
    
    return user_prefs